In [13]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb  # make sure this is installed
import pandas as pd



In [15]:
# Replace this with your cleaned CSV file path
df = pd.read_csv('gBqE3R1cmOb0qyAv.csv')  # or the name you saved


In [16]:
df_encoded = df.copy()
label_encoders = {}

for col in df_encoded.select_dtypes(include='object').columns:
    le = LabelEncoder()
    df_encoded[col] = le.fit_transform(df_encoded[col])
    label_encoders[col] = le  # Save encoder for future use if needed


In [17]:
X = df_encoded.drop('churned', axis=1)
y = df_encoded['churned']


In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [19]:
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)


C:\Users\crche\OneDrive\Documents\churn_prediction\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [22:52:58] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,'logloss'


In [20]:
y_pred = model.predict_proba(X_test)[:, 1]  # probability of churn
auc = roc_auc_score(y_test, y_pred)
print("✅ AUC-ROC Score:", auc)


✅ AUC-ROC Score: 0.8157901271354786


In [21]:
import joblib
joblib.dump(model, 'xgb_churn_model.pkl')


['xgb_churn_model.pkl']

In [22]:
pip install shap


     -------------------------------------- 544.4/544.4 kB 1.6 MB/s eta 0:00:00
  Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
     ---------------------------------------- 2.8/2.8 MB 4.1 MB/s eta 0:00:00
     ---------------------------------------- 30.3/30.3 MB 3.6 MB/s eta 0:00:00
     ---------------------------------------- 12.9/12.9 MB 4.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.0
    Uninstalling numpy-2.3.0:
      Successfully uninstalled numpy-2.3.0
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\crche\\OneDrive\\Documents\\churn_prediction\\venv\\Lib\\site-packages\\~umpy.libs\\libscipy_openblas64_-13e2df515630b4a41f92893938845698.dll'
Check the permissions.


[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
import shap

# Initialize JavaScript visualization
shap.initjs()

# Explain the model's predictions using SHAP
explainer = shap.Explainer(model)

# Select a sample of the test set for speed
shap_values = explainer(X_test)


ModuleNotFoundError: No module named 'shap'

In [ ]:
shap.summary_plot(shap_values, X_test)
